In [ ]:
import sys
sys.version

'3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]'

# Installing Libraries

In [ ]:
!pip install --upgrade pip
!pip install speechrecognition pyttsx3 wikipedia-api
!pip install pyaudio

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
D:\Anaconda Files\python.exe -m pip install --upgrade pip


In [ ]:
!pip install --upgrade wikipedia



# Importing Libraries

In [ ]:
import speech_recognition as sr
import pyttsx3
import datetime
import webbrowser
import os
import time
import subprocess
import wikipediaapi

# Text-To-Speech

In [ ]:
engine = pyttsx3.init()
voices = engine.getProperty('voices')
if voices:
    engine.setProperty('voice', voices[0].id)
engine.setProperty('rate', 180)  # speed of speech

In [ ]:
def speak(text: str) -> None:
    print(f"Assistant: {text}")
    engine.say(text)
    engine.runAndWait()

# Respond

In [ ]:
speak("Hello There, I am your voice assistant is ready.")


Assistant: Hello There, I am your voice assistant is ready.


# GREETING

In [ ]:
def wishMe() -> None:
    hour = datetime.datetime.now().hour
    if 0 <= hour < 12:
        speak("Good morning!")
    elif 12 <= hour < 18:
        speak("Good afternoon!")
    else:
        speak("Good evening!")
    speak("I am your voice activated AI chatbot. How can I help you?")


# TESTING

In [ ]:
wishMe()

Assistant: Good morning!
Assistant: I am your voice activated AI chatbot. How can I help you?


In [ ]:
def take_command():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 0.8
        r.energy_threshold = 300

        try:
            audio = r.listen(source, timeout=5, phrase_time_limit=5)
        except Exception:
            speak("I didn't hear anything. Please try again.")
            return ""

    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language="en-IN")
        print("You said:", query)
        return query.lower()

    except Exception:
        speak("Say that again please...")
        return ""


#  Wikipedia search

In [ ]:
import wikipedia
wikipedia.set_lang("en")

def wikipedia_search(query: str, sentences: int = 3) -> str:
    query = query.strip()

    try:
        # Direct summary attempt
        return wikipedia.summary(query, sentences=sentences, auto_suggest=True)

    except wikipedia.DisambiguationError as e:
        # For ambiguous terms — choose the first suggested topic
        best = e.options[0]
        summary = wikipedia.summary(best, sentences=sentences)
        return f"(Showing best match: {best})\n\n{summary}"

    except wikipedia.PageError:
        # If page not found — perform search
        results = wikipedia.search(query)
        if results:
            best = results[0]
            summary = wikipedia.summary(best, sentences=sentences)
            return f"(Closest match: {best})\n\n{summary}"
        else:
            return "I could not find anything related to your query."

    except Exception as e:
        return "Something went wrong while searching Wikipedia."


# Testing

In [ ]:
print(wikipedia_search("Who Is Bahubali"))


Baahubali: The Beginning (transl. One With Strong Arms) is a 2015 Indian Telugu-language epic action film co-written and directed by S. S. Rajamouli, and produced by Shobu Yarlagadda and Prasad Devineni under Arka Media Works. It features Prabhas in a dual role alongside Rana Daggubati, Anushka, Tamannaah Bhatia, Ramya Krishnan, Sathyaraj, and Nassar. The first of a duology of films, it follows Sivudu, an adventurous young man who helps his love Avantika rescue Devasena, the princess of Kuntala who is now a prisoner under the tyrannical rule of king Bhallaladeva.


# Search Into Your Voice Assistant

In [ ]:
def handle_command(query):
    if "hello" in query or "hi" in query:
        speak("Hello Balu, how can I help you?")

    elif "open google" in query:
        speak("Opening Google...")
        webbrowser.open("https://google.com")
        
    elif "open youtube" in query:
        speak("Opening YouTube...")
        webbrowser.open("https://youtube.com")
    elif ("wikipedia" in query
          or "who is" in query
          or "what is" in query
          or "tell me about" in query):

        speak("Searching Wikipedia...")

        cleaned = query.replace("wikipedia", "")
        cleaned = cleaned.replace("who is", "")
        cleaned = cleaned.replace("what is", "")
        cleaned = cleaned.replace("tell me about", "")
        cleaned = cleaned.strip()

        if cleaned == "":
            speak("Please tell me what to search.")
        else:
            result = wikipedia_search(cleaned, sentences=3)
            speak(result)
    elif "time" in query:
        now = datetime.datetime.now().strftime("%H:%M")
        speak(f"The time is {now}.")
    elif "date" in query:
        today = datetime.datetime.now().strftime("%B %d, %Y")
        speak(f"Today's date is {today}.")
    elif "write a note" in query or "take a note" in query:
        speak("What should I write?")
        note = take_command()
        if note:
            with open("notes.txt", "a", encoding="utf-8") as f:
                f.write(f"{datetime.datetime.now()}: {note}\n")
            speak("Note written successfully.")

    elif "stop" in query or "exit" in query or "quit" in query:
        speak("Goodbye Balu!")
        return False
    else:
        speak("I am not sure how to help with that.")
    return True


In [ ]:
def run_assistant():
    wishMe()
    running = True

    while running:
        query = take_command()
        running = handle_command(query)
        time.sleep(1)


In [ ]:
run_assistant()